#### Скачивание модулей

In [ ]:
pip install pymorphy2

In [ ]:
pip install spellchecker

In [ ]:
pip install pyspellchecker

#### **Вспомогательные [файлы](https://drive.google.com/drive/folders/1HGMld4ZghhdL0faOMVLW1ECC2xvzhWf0?usp=sharing):**

*  uniq_cases.xlsx (список с редкими словами)
*  предлоги.txt (список предлогов русского языка)
*  даль_словарь.txt (толковый словарь Даля)
*  Обучающий корпус.xlsx (размеченные данные Обучающего корпуса)
*  main_source_list.txt (токены Основного корпуса)

### **Алгоритм**

In [1]:
#@title Используем словарь (то есть сперва ищем слово в списке, а если его там нет, уже идем по алгоритму)
import re
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
from difflib import SequenceMatcher
import unicodedata
from itertools import groupby
import numpy as np
morph = MorphAnalyzer()
from spellchecker import SpellChecker
russian = SpellChecker(language='ru')

def clean_word(word):
    if '<w>' in word:
        return re.search(r'<w>(.*)</w>', word).group(1)
    else:
        return word

def part_of_speech(word):
    if ' ' not in word:
        return morph.parse(word)[0].tag.POS
    else:
        return ''
        
def lemma_func(word):
    if ' ' not in word:
        return morph.parse(word)[0].normal_form
    else:
        return ''

def tags(tag,word):
    if len(tag) == 0 and ' ' not in word:
      return morph.parse(word)[0].tag.POS
    elif ' ' in word:
      return ''
    else:
      tags = {'существительное': 'NOUN',
              'прилагательное': 'ADJF',
              'наречие': 'ADVB',
              'междометие': 'INTJ',
              'предлог': 'PREP'
              }
      if tag in tags.keys():
        return tags[tag]
      else:
        return tag
        
df1 = pd.read_excel('Обучающий корпус.xlsx', sheet_name='distinct')
df2 = pd.read_excel('Обучающий корпус.xlsx', sheet_name='оставить')
uniq_cases = pd.read_excel('uniq_cases.xlsx')

with open('предлоги.txt', 'r') as file:
    prepos = [line.rstrip() for line in file]
with open('даль_словарь.txt', 'r', encoding='utf-8') as file:
    lines = [line.rstrip() for line in file]
dal_voc = []
for line in lines:
    if re.search(r'^[А-Я]+', line):
      word = re.match(r'^[А-Я]+', line).group(0).lower()
      if len(word) > 1:
          dal_voc.append(word)

new_df = pd.DataFrame()
new_df['Было'] = pd.concat([df1['Было'], df2['Token']], ignore_index=True)
new_df['Станет'] = pd.concat([df1['Станет'], df2['Token']], ignore_index=True)
new_df['Частота'] = pd.concat([df1['Freq(School_Corpus)'], df2['Freq(School_Corpus)']], ignore_index=True)
new_df['Станет'] = new_df['Станет'].apply(clean_word)
vocab = dal_voc + list(set(new_df['Станет'].tolist()))
two_tokens = {'Во-о-оза-ра-за-за-за': 'Во$зараза'} #список слов с двумя токенами можно дополнять

def get_corrected1(token):
    normalized_word = unicodedata.normalize('NFKC', token)
    cleaned_token = ''.join(c for c in normalized_word if not unicodedata.combining(c))
    for l in cleaned_token:
        if l not in 'АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоПпРрСсТтУуФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя-':
            cleaned_token = cleaned_token.replace(l, '')
    cleaned_token = re.sub(r'(-)\1+', r'\1', cleaned_token)
    def matchsubstring(m,n):
        common = []
        while True:
            seqMatch = SequenceMatcher(None,m,n)
            match = seqMatch.get_matching_blocks()
            if len(match) > 1:
                for i in range(len(match)-1):
                    common.append(m[match[i].a:match[i].a+match[i].size])
                if len(n) != match[-2].size+1 and len(m) != match[-2].a+match[-2].size:
                    m = m[match[-2].a+match[-2].size:]
                else:
                    if m[-1] == n[0] and n[0] not in common:
                        common.append(n[0])
                    break
            else:
                break
        return common

    if token in vocab or token.lower() in vocab:
        return [token, '']

    elif len(token.split('-')) == 2 and token.split('-')[0] in vocab and token.split('-')[1] in vocab and len(token.split('-')[0]) >=4 and len(token.split('-')[1]) >= 4: #редкие сложные сущ.
        return [token, '']

    elif len(cleaned_token.split('-')) == 2 and morph.parse(cleaned_token.split('-')[0])[0].normal_form in vocab and morph.parse(cleaned_token.split('-')[1])[0].normal_form in vocab:
        return [cleaned_token, '']

    elif token.replace('-', '').lower() in vocab or token.replace('-', '').lower() in vocab: #случаи скандирования
        return [token.replace('-', '').lower(), '']

    elif token in prepos:
        return [token, 'предлог']

    elif re.sub(r'((\w)(-)?)\1+', r'\1', token.lower().replace('-', '')) in vocab and len(re.sub(r'((\w)(-)?)\1+', r'\1', token.lower().replace('-', ''))) > 3: #случаи скандирования
        token = re.sub(r'((\w)(-)?)\1+', r'\1', token.lower().replace('-', ''))
        if cleaned_token[0].isupper():
            token = token.capitalize()
        return [token, '']

    elif re.sub(r'((\w)(-)?)\1+', r'\1', token.replace('-', '')) in vocab and len(re.sub(r'((\w)(-)?)\1+', r'\1', token.replace('-', ''))) > 3: #случаи скандирования
        token = re.sub(r'((\w)(-)?)\1+', r'\1', token.replace('-', ''))
        if cleaned_token[0].isupper():
            token = token.capitalize()
        return [token, '']

    elif token.lower() in uniq_cases['Было'].tolist():
        token = uniq_cases[uniq_cases['Было'] == token.lower()]['Станет'].item()
        if cleaned_token[0].isupper():
            token = token.capitalize()
        if cleaned_token[cleaned_token.find('-')+1].isupper():
            i = token.find('-')+1
            token = list(token)
            token[i] = token[i].upper()
            token = ''.join(token)
        return [token, '']

    elif len(token.split('-')) == 2 and token.split('-')[1] in vocab and token.split('-')[0][0].isupper() and len(token.split('-')[0]) >=4 and len(token.split('-')[1]) >= 4: #случаи вида Машка-одноклассница
        return [token, '']

    elif (len(token.split('-')) == 2 and morph.parse(token.split('-')[1])[0].normal_form in vocab and len(token.split('-')[0]) > 1 and len(token.split('-')[0]) <=3 and token.split('-')[0].isupper() and len(token.split('-')[1]) >= 4) or re.search(r'^АБВГД-ейк', token): #ищем аббревиатуры вида ВИЧ-инфекция, АБ-центр
        return [token, '']

    elif len(token.split('-')) == 2 and token.split('-')[0] in ['аль', 'Аль'] and (morph.parse(token.split('-')[1])[0].normal_form in vocab or token.split('-')[1][0].isupper()):
        return [token, '']

    elif token.split('-')[0] in ['альт', 'Альт'] and len(token.split('-')) == 2 and len(token.split('-')[1]) > 3:
        return [token, '']

    elif token.split('-')[0] in ['альт', 'Альт'] and len(token.split('-')) == 3 and token.split('-')[1] == 'а' and len(token.split('-')[2]) > 3:
        return [token, '']

    elif token in two_tokens.keys():
        return [two_tokens[token], '']


    else:
        cleaned_token = cleaned_token.replace(' ', '')
        cleaned_token = cleaned_token.replace('.', '')
        word = re.sub(r'(-)\1+', r'\1', cleaned_token)
        result = []

        if len(word.split('-')) == 2: #если в слове только один дефис
            if re.search(r'[А-Яа-яЁё]{3,}(?<!то)-то$', word):
                if re.search(r'ой|ый|ий|ая|яя|ое|ее|ые|ие|ого|его|ому|ему|ую|юю|им|ым|ом|ем|ых|их|ыми|ими-то$', word): #если слово имеет окончание прилагательного
                    result = [word, 'прилагательное']
                else:
                    result = [word, '']
            if re.search(r'^[А-Я][а-я]+-([А-Я][а-я]+-)?[А-Я][а-я]+$', word): #если слово и его части, разделенные дефисом, начинаются с заглавной буквы
                #преобразования не нужны, т.к. это имя собственное
                if re.search(r'ой|ый|ий|ая|яя|ое|ее|ые|ие|ого|его|ому|ему|ую|юю|им|ым|ом|ем|ых|их|ыми|ими$', word): #если слово имеет окончание прилагательного
                    result = [word, 'прилагательное']
                else:
                    result = [word, 'существительное']
            elif re.search(r'^(кое|кой|койни)-(где|как|когда|куда)$', word, re.IGNORECASE): #наречия вида кое-где, кое-когда и т.д.
                if 'койни' in word:
                    word = word.replace('койни', 'кое')
                result = [word, 'наречие']
            elif re.search(r'^(кое|кой|койни)-(что|кто)$', word.lower()): #местоимения вида кое-кто в И.п. и В.п.
                if 'койни' in word:
                    word = word.replace('койни', 'кое')
                result = [word, 'местоимение']
            elif re.search(r'^(кое|кой|койни)-как(ой|ая|ое|ие|ого|их)$', word, re.IGNORECASE):  #местоимения вида кое-кто в косвенных падежах
                if 'койни' in word:
                    word = word.replace('койни', 'кое')
                result = [word, 'местоимение']
            elif re.search(r'^(кто|что|че|чё|как|куда|сколько|когда|где|гдей|чтой|ктой|кого|кому|кого|кем|ком|чей|чего|чево|чиво|чему|чем|чём|чей|чья|чьё|чьи|чьего|чьей|чьих|чьему|чьим|чью|чьим|чьими|чьём)-(то|либо|нибудь|нить|нють|нито)$', word.lower()):
                if re.search(r'^(чё|че|чтой|што|чо)-(то|либо|нибудь|нить|нють|нито)$', word.lower()):
                    word = re.sub(r'(че|чё|чтой|што|чо)', r'что', word.lower())
                elif 'чево' in word.lower() or 'чиво' in word.lower():
                    word = re.sub(r'(чево|чиво)', r'чего', word.lower())
                elif 'гдей' in word.lower():
                    word = word.replace('гдей', 'где')
                elif word.startswith('ктой') or word.startswith('Ктой'):
                    word = word.replace('ктой', 'кто'); word = word.replace('Ктой', 'Кто')
                if 'нють' in word.lower() or 'нито' in word.lower():
                    word = re.sub(r'нють|нито', r'нибудь', word, re.IGNORECASE)
                if 'как' in word or 'где' in word or 'куда' in word or 'когда' in word:
                    result = [word, 'наречие']
                else:
                    result = [word, 'местоимение']
            elif re.search(r'^(как|так)(ой|ая|ое|ие|ого|ово|ой|их|ому|им|ую|ою|ими|ом|у)-(то|ту|либо|нибудь|нить)$', word, re.IGNORECASE): #местоимения вида какой-либо, такой-то
                if 'каково' in word.lower():
                    word = re.sub(r'каково', 'какого', word); word = re.sub(r'Каково', 'Какого', word)
                elif 'таку' in word.lower():
                    word = re.sub(r'таку', 'такую', word); word = re.sub(r'Таку', 'Такую', word)
                elif 'каку' in word.lower():
                    word = re.sub(r'каку', 'какую', word); word = re.sub(r'Каку', 'Какую', word)
                if 'ту' in word.lower():
                    word = re.sub(r'ту', 'то', word)
                result = [word, 'местоимение']

            elif re.search(r'^[А-Яа-яЁё]{2,}(о|е)-[а-яё]{2,}(ый|ой|ая|ое|ые|ие|ье|ей|ую|ого|ому|ым|им|ом|их|ее|ими|ых|его|ий|его|ему|им|ем|ыми)$', word, re.IGNORECASE): #прилагательные типа сине-красный
                if not re.search(r'о-о', word) and not re.search(r'е-е', word): #исключаем случаи, когда это не составное прил., а растянутая буква в середине слова
                    result = [word, 'прилагательное']

            elif re.search(r'^по-[а-я]{3,}(ьи|ски|цки|ому|ему)$', word, re.IGNORECASE) or re.search(r'^по-(тво|мо|ихн)ему$', word, re.IGNORECASE) or re.search(r'^по-иному$', word, re.IGNORECASE): #наречия типа по-волчьи
                result = [word, 'наречие']

            elif re.search(r'^в-[^в][а-я]{2,}(ых|их)$', word, re.IGNORECASE) or re.search(r'^во-[а-я]{3,}(ых|их)$', word, re.IGNORECASE) or word == 'в-восьмых': #штуки типа во-первых
                result = [word, 'наречие']

            elif re.search(r'^(северо|юго)-(восток|запад)(а|у|ом|е)?$', word, re.IGNORECASE): #северо-запад, юго-восток
                result = [word, 'существительное']

            elif re.search(r'^(норд|зюйд)-(ост|вест)', word, re.IGNORECASE): #добавить сюда норд, вест
                if re.search(r'(ост|вест)(а|у|ом|е)?$', word, re.IGNORECASE): #зюйд-вест
                    result = [word, 'существительное']
                elif re.search(r'остов|вестов', word, re.IGNORECASE): #норд-остовый (можно добавить перечень окончаний прил.)
                    result = [word, 'прилагательное']

            elif re.search(r'^(авто|агро|аэро|био|вело|гидро|зоо|кино|макро|микро|метео|мото|нео|радио|теле|стерео|фото|электро)-[а-я]{3,}$', word, re.IGNORECASE):
                word = word.replace('-','') #с такими приставками не нужен дефис, удаляем его
                result = [word, 'существительное']

            elif re.search(r'^(вице|лейб|обер|унтер|ундер|штаб|экс)-[а-я]{3,}$', word, re.IGNORECASE):
                if 'ундер' in word:
                    word = word.replace('ундер', 'унтер')
                result = [word, 'существительное']

            elif re.search(f'b/пол-[аеёиоуыэюя][а-я]+$', word) or re.search(f'^пол-л[а-я]+$', word) or re.search(f'^пол-[А-ЯЁ][а-я]+$', word):
                result = [word, 'существительное']

        if len(result) == 0: #если у нас нет результата

            #подсчет согласных и гласных
            conson = [x for x in list(word.lower()) if x in 'бвгджзклмнпрстфхцчшцьъ']
            vowels = [x for x in list(word.lower()) if x in 'ауоыиэяюёе']

            if len(set(conson)) == 0 or len(set(vowels)) == 0: #если слово состоит только из гласных или только из согласных
                result = [word, 'междометие']
                return result

            elif len(set(conson)) == 1 and len(set(vowels)) == 1: #нн-нее ооо-ох
                if len([''.join(g) for _, g in groupby(word.lower().replace('-', ''))]) == 2:
                    # word = [''.join(g) for _, g in groupby(word.lower().replace('-', ''))][0][0] + [''.join(g) for _, g in groupby(word.lower().replace('-', ''))][1][0] нужно ли удалять повторы в словах типа о-о-ох
                    word2 = re.sub(r'((\w)(-)?)\1+', r'\1', word.lower().replace('-', ''))
                    if word2 in ['ты', 'вы', 'мы', 'он', 'их', 'им', 'ей']:
                        result = [word2, 'местоимение']
                    elif word2 in ['че', 'чё','чо', 'шо']:
                        result =  ['что', '']
                    elif word2 in ['не', 'но', 'ни', 'да', 'те', 'се', 'то', 'ну']:
                        result = [word2, '']
                    elif word2 in prepos:
                        result = [word2, 'предлог']
                    elif len([''.join(g) for _, g in groupby(word.replace('-', ''))]) == 2:
                        result = [word, 'междометие']

            elif len(set(conson)) == 1 and len(set(vowels)) == 2: #ура-аа и проч. межд. c двумя гласными и согласной между ними вау
                if len([''.join(g) for _, g in groupby(word.replace('-', ''))]) == 3 and word[0] in 'ауоыиэяюёе' and word[-1] in 'ауоыиэяюёе':
                    word = [''.join(g) for _, g in groupby(word.replace('-', ''))][0][0] + [''.join(g) for _, g in groupby(word.replace('-', ''))][1][0] + [''.join(g) for _, g in groupby(word.replace('-', ''))][2][0]
                    if word in ['оно', 'они', 'ему']:
                        result = [word, 'местоимением']
                    elif word in prepos:
                        result = [word, 'предлог']
                    else:
                        result = [word, 'междометие']

            elif len(set(conson)) == 2 and len(set(vowels)) == 1:
                word2 = [''.join(g) for _, g in groupby(word.replace('-', ''))][0][0] + [''.join(g) for _, g in groupby(word.replace('-', ''))][1][0] + [''.join(g) for _, g in groupby(word.replace('-', ''))][2][0]
                if word2 in ['мне']:
                    result = [word2, 'местоимением']

            if len(result) == 0:
              sylls = list(map(lambda x: re.sub(r'-?(\w)-?\1+', r'\1', x, re.IGNORECASE), word.lower().split('-'))) #делим слово на части и удаляем подряд идущие буквы: Ниии-ннни --> ни-ни
              if len(sylls) == 2:
                  if sylls[0] == sylls[1] and len(sylls[0]) == 2: #если слово состоит из двух одинаковых частей длины 2, то оно может быть как междометием (ха-ха), так и сущ.
                    word2 = re.sub(r'((\w)(-)?)\1+', r'\1', word.lower().replace('-', ''))
                    if word2 in ['папа', 'мама', 'дядя', 'няня', 'баба']:
                        return [word2, 'существительное']
                    else:
                        return [word, 'междометие']
                  elif sylls[0] == sylls[1]:
                      return [word, ''] #слова типа "только-только"
              elif len(sylls) > 2 and len(set(sylls)) == 1: #если слово состоит из трех и более одинаковых слогов, то это, скорее всего, междометие
                  result = [word, 'междометие']
                  return result
              elif sylls[0] in 'ауоыиэяюёе' and len(set(sylls[1:])) == 1: #междометия, которые начинаются с одной гласной, а остальные его части одинаковые: э-ге-ге, о-хо-хо
                  result = [word, 'междометие']
                  return result

            if re.search(r'^ы[бвгджзклмнпрстфхцчшцьъ]', word, re.IGNORECASE): #если слово начинается с Ы, после которой следует согласный
                result = [word, 'междометие']
                return result

            word2 = re.sub(r'-?(\w)-?\1+', r'\1', word.replace('-', ''), re.IGNORECASE) #удаляем подряд идущие буквы: Ктооо-то --> ктото
            if re.search(r'^(кто|что|че|чё|чо|шта|шо|как)$', word2, re.IGNORECASE): #восклицания вида Чтооо Каак Ктоо
                if re.search(r'че|чё|чо|чтой|шта|што|шо', word2.lower()):
                    word2 = re.sub(r'(че|чё|чо|чтой|шта|што|шо)', r'что', word2.lower())
                if word2.lower() == 'как':
                    result = [word2, 'наречие']
                else:
                    result = [word2, 'местоимение']
            elif re.search(r'^(кто|что|че|чё|как|куда|когда|где|гдей|чтой|ктой|кого|кому|кого|кем|ком|чей|чего|чево|чиво|чему|чем|чём|чей|чья|чьё|чьи|чьего|чьей|чьих|чьему|чьим|чью|чьим|чьими|чьём)(то|либо|нибудь|нить|нють|нито)$', word2.lower()):
                if re.search(r'че|чё|чо|чтой|шта|шо', word2.lower()):
                    word = re.sub(r'(че|чё|чо|чтой|шта|што|шо)', r'что', word.lower())
                for part in ['то', 'либо', 'нибудь', 'нить', 'нито']:
                    if word2.endswith(part):
                        if 'гдей' in word2:
                            word2 = word2.replace('гдей', 'где')
                        if word2.startswith('чтой'):
                            word2 = word2.replace('чтой', 'что')
                        elif word2.startswith('ктой'):
                            word2 = word2.replace('ктой', 'кто')
                        if part == 'то' and word2[-2:] == 'то': #исключаем случаи постановки дефиса перед первым -то (кто-то не должно переходить в к-то-то)
                            word2 = re.sub(r'то$', '-то', word2)
                        elif part == 'нють':
                            word2 = word2.replace('нють', '-нибудь')
                        elif part == 'нито':
                            word2 = word2.replace('нито', '-нибудь')
                        else:
                            word2 = word2.replace(part, f'-{part}')
                        if 'как' in word2 or 'куда' in word2 or 'когда' in word2 or 'где' in word2:
                            result = [word2, 'наречие']
                        else:
                            result = [word2, 'местоимение']
                        break

            elif re.search(r'^(кое|кой|койни)(где|как|когда|куда)$', word2, re.IGNORECASE):
                if 'койни' in word2:
                    word2 = word2.replace('койни', 'кое')
                for part in ['где', 'как', 'когда', 'куда']:
                    if part in word2:
                        word2 = word2.replace(part,  f'-{part}')
                        break
                result = [word2, 'местоимение']

            elif re.search(r'^(кое|кой)(что|кто|чего|кого|чему|кому|чем|кем})$', word2, re.IGNORECASE):
                word2 = re.sub(r'кое|кой|койни', 'кое-', word2, re.IGNORECASE) #выбираем именно 1ую часть, т.к. она не склоняется
                result = [word2, 'местоимение']

            elif re.search(r'^(кое|кой|койни)как(ой|ая|ое|ие|ого|их|у)$', word2, re.IGNORECASE):
                if 'каку' in word2.lower():
                    word2 = re.sub(r'каку', 'какую', word2)
                word2 = re.sub(r'кое|кой|койни', 'кое-', word2, re.IGNORECASE)
                result = [word2, 'местоимение']

            elif re.search(r'^[А-Яа-яЁё]{2,}о-([А-Яа-яЁё]{2,}(о|е)-)?[А-Яа-яЁё]{2,}(ый|ой|ая|ое|ые|ие|ье|ей|ую|ого|ому|ым|им|ом|их|ее|ими|ых|его|ий|его|ему|им|ем|ыми)$', word, re.IGNORECASE): #штуки типа желто-сине-красный или синеее-ее-желтый
                result = [word, 'прилагательное']

            elif len(word.split('-')) == 2:
                if len(word.split('-')[0]) > 4 and len(word.split('-')[1])> 4:
                    substr = matchsubstring(word.split('-')[0], word.split('-')[1])
                    flag = 0
                    #если слова не пересекаются
                    for i in range(len(substr)):
                        if word[0].endswith(substr[i]) and word[1].startswith(substr[i]): #если конец одного слога = начало другого
                            flag = 1
                            break

                    if word[0][-1] == word[1][0]:
                        flag = 1

                    if flag == 0:
                        result = [word, '']
                        return result

        if len(result) != 0:
            if cleaned_token[0].isupper(): #сохраням исходный регистр
                result[0] = result[0].capitalize()
            if cleaned_token[cleaned_token.find('-')+1].isupper():
                i = result[0].find('-')+1
                result[0] = list(result[0])
                result[0][i] = result[0][i].upper()
                result[0] = ''.join(result[0])
            return result
        else:
            word = word.lower()
            word = word.split('-')

        if len(word) > 1:
            syllables = []
            for k in range(0, len(word)-1): #проходимся по слогам
                if k == 0:
                    syllables = [word[k]] #сразу записываем первый слог в список отфильтрованных слогов
                substr = matchsubstring(word[k], word[k+1])
                flag = 0
                #исключаем часть слога, которая является общей для двух слогов
                for i in range(len(substr)):
                    if word[k].endswith(substr[i]) and word[k+1].startswith(substr[i]): #если конец одного слога = начало другого
                        if word[k+1][len(substr[i])-1:] != syllables[-1] or (word[k+1][len(substr[i])-1:] != syllables[-1] and word[k+1] in ['па', 'ма', 'те', 'тё', 'дя', 'ня', 'ба']): #если слоги не одинаковые
                            if len(word[k+1][len(substr[i]):]) > 0: #если оставшаяся длина слова больше 0
                                syllables.append(word[k+1][len(substr[i]):])
                            flag = 1

                #последняя буква первого слога = первая буква второго слога
                if flag == 0:
                    if len(word[k]) > 1 and len(word[k+1]) and word[k][-1] == word[k+1][0]:
                        if word[k+1][len(substr[i])-1:] != syllables[len(syllables)-1]:
                            if len(word[k+1][1:]) > 1:
                                syllables.append(word[k+1][1:])
                                flag = 1
                    else:
                        #не добавляем слог, если предыдущий такой же
                        if word[k+1] != syllables[len(syllables)-1]:
                            syllables.append(word[k+1])
                        flag = 1
            #4 шаг
            syllables2 = []
            i = 0
            while i < len(syllables): #склеиваем одинарные слоги
                if len(syllables[i]) == 1: #если слог состоит из одной буквы
                    if i != len(syllables)-1: #если этот слог не последний
                        if len(syllables[i+1]) == 1: #если следующий слог состоит из одной буквы
                            syllables2.append(syllables[i]+syllables[i+1]) #склеиваем подряд идущие слоги, состоящие из одной буквы
                            i += 2
                        else:
                            syllables2.append(syllables[i]) #если следующий слог не одинарный, то просто добавляем текущий к слову
                            i += 1
                    else:
                        syllables2.append(syllables[i])
                        i += 1
                else:
                    syllables2.append(syllables[i])
                    i += 1

            syllables3 = []
            for i in range(0,len(syllables2)):
                if i != len(syllables2)-1:
                    if syllables2[i] != syllables2[i+1] and not syllables2[i+1].startswith(syllables2[i]):
                        syllables3.append(syllables2[i])
                else:
                    syllables3.append(syllables2[i])

            #5 шаг
            new_word = ''.join(syllables3)
            letters = [new_word[0]]
            for i in range(1,len(new_word)):
                if new_word[i] != new_word[i-1]:
                    letters.append(new_word[i])
            new_word = ''.join(letters)
        else:
            letters = [word[0][0]]
            for i in range(1,len(word[0])):
                if word[0][i] != word[0][i-1]:
                    letters.append(word[0][i])
            new_word = ''.join(letters)


        #восстанавливаем окончание имен, состоящее из двух гласных букв
        if cleaned_token[-1] == cleaned_token[-2]:
            if re.search(r'(.)*[бвгдзклмнпрстфх]юю$', cleaned_token) or re.search(r'(.)*[бвгджзклмнпрстфхцчшщ]ее$', cleaned_token) \
               or re.search(r'(.)*[бвгджзклмнпрстфхцчшщ]ии$', cleaned_token) or re.search(r'(.)*[бвгдзклмнпрстфх]яя$', cleaned_token): #если две последние буквы = допустимые окончания из двух одинаковых гласных
                new_word += new_word[-1]

        #6 шаг
        replacements = list(set(re.findall(r'\w[ьъ]\w', new_word))) #находим такие сочетания, где ь или ъ находятся между двумя согласными, что недопустимо
        for match in replacements:
            if match[0] == match[2]: #если первая две согласные одинаковые
                if match[0] in 'лнж': #если согласная из списка лнж = согласные, которые могут удваиваться
                    new_word = re.sub(match, match[0]+match[2], new_word, flags=re.IGNORECASE) #заменяем льл на лл и т.д.
                else:
                    new_word = re.sub(match, match[0], new_word, flags=re.IGNORECASE)
        #7 шаг
        doubled = ['ёё', 'йй', 'шш', 'щщ', 'ъъ', 'ыы', 'ьь', 'ээ', 'аа']
        replace_letters = {  'йа': 'я',
                            'йюу': 'ю',
                            'йю': 'у',
                            'чщ': 'ч',
                            'йе': 'е',
                            'оа': 'о',
                            'еэ': 'е',
                            'юу': 'ю',
                            'ёо': 'ё',
                            'иы': 'и',
                            'йё': 'ё', #йёлки
                            'ийя': 'я',
                            'иы': 'и',
                            'йя': 'я'
        }

        if new_word.endswith('ыи'):
            new_word = new_word[:-1]
        elif new_word.endswith('йсь'):
            new_word = re.sub(r'ь$', 'я', new_word)
        elif new_word.endswith('ёа'):
            new_word = re.sub(r'ёа$', 'ё', new_word)
        elif new_word.endswith('лася'):
            new_word = re.sub(r'ся$', 'сь', new_word)
        if re.search(r'е(-)?(э(-)?){2,}$', cleaned_token): #ты-и
            new_word = re.sub(r'е(-)?(э(-)?){1,}$','е', new_word)
        elif new_word.endswith('еэ') and len(new_word) > 3:
            new_word = re.sub(r'еэ$', 'ей', new_word) #?
        elif new_word.endswith('уа'):
            if re.search(r'[А-Яа-яЁё]{2,}у{1,}уа(а)?$', cleaned_token):
                new_word = re.sub(r'уа$', 'у', new_word)
        if re.search(r'е(-)?(о(-)?){2,}$', cleaned_token):
            new_word = re.sub(r'е(-)?(о(-)?){1,}$','е', new_word)
        if re.search(r'ы(-)?(и(-)?){2,}$', cleaned_token): #ты-и
            new_word = re.sub(r'ы(-)?(и(-)?){1,}$','ы', new_word)
        if re.search(r'нн(ый|ой|ая|ое|ые|ие|ье|ей|ую|ого|ому|ым|им|ом|их|ее|ими|ых|его|ий|его|ему|им|ем|ыми)', new_word): #восстанавливаем удвоенную согласную "нн"
            index = new_word.rfind('н')
            new_word = new_word[:index] + 'н' + new_word[index:]
        for el in doubled:
            if el in new_word:
                new_word = new_word.replace(el, el[0])

        for k in replace_letters.keys():
            new_word = new_word.replace(k, replace_letters[k])

        #8 шаг
        replacements = list(set(re.findall('[аеёиоуыэюя][ьъ][аеёиоуыэюя]', new_word)))
        for match in replacements:
            new_word = re.sub(match, match[1], new_word, flags=re.IGNORECASE)

        #9 шаг
        replacements = list(set(re.findall('[я][аеёиоуыэю]', new_word)))
        for match in replacements:
            if match == 'яю':
                if not new_word.endswith(match):
                    new_word = re.sub(match, match[0], new_word, flags=re.IGNORECASE)
            else:
                new_word = re.sub(match, match[0], new_word, flags=re.IGNORECASE)

        if new_word.lower() in uniq_cases['Было'].tolist():
            new_word = uniq_cases[uniq_cases['Было'] == new_word.lower()]['Станет'].item()

        if cleaned_token[0].isupper():
            new_word = new_word.capitalize()

        lemma1 = morph.parse(new_word)[0].normal_form
        lemma2 = morph.parse(new_word)[0].normal_form.capitalize()
        if new_word.lower() not in uniq_cases['Было'].tolist() and new_word.lower() not in vocab:
            if lemma1 not in vocab or lemma2 not in vocab:
                if russian.correction(new_word) is not None:
                    new_word = russian.correction(new_word)

    return [new_word, '']

C:\Users\602\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\602\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\602\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
C:\Users\602\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_m

### **Разметка**

In [1]:
def tags(tag,word):
    if len(tag) == 0 and ' ' not in word:
      return morph.parse(word)[0].tag.POS
    elif ' ' in word:
      return ''
    else:
      tags = {'существительное': 'NOUN',
              'прилагательное': 'ADJF',
              'наречие': 'ADVB',
              'междометие': 'INTJ',
              'предлог': 'PREP'
              }
      if tag in tags.keys():
        return tags[tag]
      else:
        return tag

In [2]:
#путь к файлу с вашими токенами
# tokens_path = "<ваш список токенов>.txt"

#для примера взят "source_list.txt из нашего репозитория"
tokens_path = "source_list.txt"

In [4]:
main = []
with open(tokens_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    for line in lines:
      if '-' in line:
          main.append(line.replace('\n', ''))


data = pd.DataFrame(columns = ['Было', 'Станет', 'UPOS', 'Лемма'])
for x in main:
    res = get_corrected1(x)
    if res[1] == 'междометие':
        lemma = res[0]
    else:
        lemma = lemma_func(res[0])
    data.loc[ len(data.index )] = [x, res[0], tags(res[1], res[0]), lemma]

sample_df = pd.DataFrame(columns = ['Было', 'Станет', 'UPOS', 'Лемма'])
for index, row in data.iterrows():
    # print(row['Было'])
    old = row['Было']; new = row['Станет']
    if row['UPOS'] == 'INTJ':
        sample_df.loc[ len(sample_df.index )] = [f'<w>{old}</w>', f'<distinct form="{old}"><w>{new}</w></distinct>', row['UPOS'], row['Было']]
    else:
        lemma = row['Лемма']
        if row['Станет'][0].isupper():
            lemma = list(lemma)
            lemma[0]= lemma[0].upper()
            lemma = ''.join(lemma)
        token_index = [match.start() for match in re.finditer('-', row['Станет'])]
        lemma_index = [match.start() for match in re.finditer('-', lemma)]
        k = 0
        if len(token_index) != 0 and len(lemma_index) != 0:
            for i in token_index:
                if i < len(lemma)-1:
                    if row['Станет'][i+1].isupper():
                        lemma = list(lemma)
                        lemma[lemma_index[k]+1] = lemma[lemma_index[k]+1].upper()
                        lemma = ''.join(lemma)
                k += 1
        if len(lemma.split('-')) == 2:
            lemma = lemma.split('-')
            if row['Было'].split('-')[0].isupper():
                lemma[0] = lemma[0].upper()
            if row['Было'].split('-')[1].isupper():
                lemma[1] = lemma[1].upper()
            lemma = '-'.join(lemma)

        sample_df.loc[len(sample_df.index )] = [f'<w>{old}</w>', f'<distinct form="{old}"><w>{new}</w></distinct>', row['UPOS'], lemma]
sample_df.to_csv('sample_df.csv', encoding='utf-8')

In [5]:
sample_df

,Было,Станет,UPOS,Лемма
0,<w>Усть-Ильинск</w>,"<distinct form=""Усть-Ильинск""><w>Усть-Ильинск<...",NOUN,Усть-Ильинск
1,<w>Урааа-а</w>,"<distinct form=""Урааа-а""><w>Ура</w></distinct>",INTJ,Урааа-а
2,<w>маа-маа</w>,"<distinct form=""маа-маа""><w>мама</w></distinct>",NOUN,мама
3,<w>светло-желтый</w>,"<distinct form=""светло-желтый""><w>светло-желты...",ADJF,светло-жёлтый
4,<w>какому-то</w>,"<distinct form=""какому-то""><w>какому-то</w></d...",ADJF,какой-то
